In [1]:
import numpy as np
import tensorflow as tf
import warnings
import helper
warnings.filterwarnings('ignore')

In [2]:
data_dir = 'data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
print(text[:81])
text = text[81:] # Remving the first line

[YEAR DATE 1989] © Twentieth Century Fox Film Corporation. All rights reserved.




In [3]:
view_sentence_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




**Implementing Preprocessing Functions** 

The first thing to do to any dataset is preprocessing. Implementing the following preprocessing functions below:
    - Lookup Table
    - Tokenize Punctuation

In [5]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    param: 
     - text: The text of tv scripts split into words
     - return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab = set(text)
    vocab_to_int = {v: k for k, v in enumerate(vocab)}
    int_to_vocab = {v: k for k, v in vocab_to_int.items()}
    
    return (vocab_to_int, int_to_vocab)

tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


**Tokenize Punctuation**

Creating a dictionary for the following symbols where the symbol is the key and value is the token:

Period ( . )

Comma ( , )

Quotation Mark ( " )

Semicolon ( ; )

Exclamation mark ( ! )

Question mark ( ? )

Left Parentheses ( ( )

Right Parentheses ( ) )

Dash ( -- )

Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it. This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. 

In [6]:
def token_lookup():
    """
    return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    token_dict = {
        
        ".": "||Period||",
        ",": "||Comma||",
        "\"": "||Quotation||",
        ";": "||Semicolon||",
        "!": "||Exclamation_Mark||",
        "?": "||Question_Mark||",
        "(": "||Left_Paranthesis||",
        ")": "||Right_Paranthesis||",
        "--" : "||Dash||",
        "\n" : "||Return||"
    }
    return token_dict

tests.test_tokenize(token_lookup)

Tests Passed


**Preprocess all the data and save it**

Running the code cell below will preprocess all the data and save it to file.

In [7]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [8]:
# The checkpoint to resume the next time
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

**Checking the TensorFlow Version**

In [9]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


**Input**

*Implementing the get_inputs() function to create TF Placeholders for the Neural Network.*

Input text placeholder named "input" using the TF Placeholder name parameter

    :Targets placeholder
    :Learning Rate placeholder
    :Returning the placeholders in the following tuple (Input, Targets, LearningRate)

In [10]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(tf.int32, shape=(None, None), name="input")
    targets = tf.placeholder(tf.int32, shape=(None, None), name="Target")
    learning_rate = tf.placeholder(tf.float32, name="LearningRate")
    
    return (inputs, targets, learning_rate)

tests.test_get_inputs(get_inputs)

Tests Passed


**Building RNN Cell and Initialize**

- Stack one or more BasicLSTMCells in a MultiRNNCell.

- The Rnn size should be set using rnn_size

- Initalize Cell State using the *MultiRNNCell's* zero_state() function

- Apply the name "initial_state" to the initial state using tf.identity()

- Return the cell and initial state in the following tuple (Cell, InitialState)

In [11]:
def get_init_cell(batch_size, rnn_size):
    """
    :param: 
        - batch_size: Size of batches
        - param rnn_size: Size of RNNs
        - return: Tuple (cell, initialize state)
    """
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=0.5)
    
    cell = tf.contrib.rnn.MultiRNNCell([drop] * 2)
    
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name='initial_state')
    
    return (cell, initial_state)

tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Creating embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [12]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    :param:
        - input_data: TF placeholder for text input.
        - vocab_size: Number of words in vocabulary.
        - embed_dim: Number of embedding dimensions
    
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)

    return embed

tests.test_get_embed(get_embed)

Tests Passed


### Building the Neural Network

In [13]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    output, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name="final_state")
    
    return (output, final_state)

tests.test_build_rnn(build_rnn)

Tests Passed


In [14]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param:
        - cell: RNN cell
        - rnn_size: Size of rnns
        - input_data: Input data
        - vocab_size: Vocabulary size
        - embed_dim: Number of embedding dimensions
    """
    embeddings = get_embed(input_data, vocab_size, embed_dim)
    
    rnn_output, f_state = build_rnn(cell, embeddings)
    
    logits = tf.contrib.layers.fully_connected(rnn_output, vocab_size, activation_fn=tf.nn.relu)
    
    return (logits, f_state)

tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implementing `get_batches` to create batches of input and targets using `int_text`.  The batches are Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [15]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    param:
        - int_text: Text with the words replaced by their ids
        - batch_size: The size of batch
        - seq_length: The length of sequence
    return: 
        - Batches as a Numpy array
    """
    n_batches = int(len(int_text) / (batch_size * seq_length))

    input_data = np.array(int_text[: n_batches * seq_length * batch_size])
    target_data = np.array(int_text[1: n_batches * seq_length *  batch_size + 1])
    
    input_batch = np.split(input_data.reshape(batch_size, -1), n_batches, 1)
    target_batch = np.split(target_data.reshape(batch_size, -1), n_batches, 1)
    
    target_batch[-1][-1][-1] = 0
    
    batches = list(zip(input_batch, target_batch))
    batches = np.array(batches)
    
    return batches

tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [28]:
# Number of Epochs
num_epochs = 800
# Batch Size
batch_size = 64
# RNN Size
rnn_size = 700
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 165
# Learning Rate
learning_rate = .003
# Show stats for every n number of batches
show_every_n_batches = 50

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/6   train_loss = 8.822
Epoch   8 Batch    2/6   train_loss = 6.034
Epoch  16 Batch    4/6   train_loss = 6.004
Epoch  25 Batch    0/6   train_loss = 5.805
Epoch  33 Batch    2/6   train_loss = 5.689
Epoch  41 Batch    4/6   train_loss = 5.638
Epoch  50 Batch    0/6   train_loss = 5.485
Epoch  58 Batch    2/6   train_loss = 5.384
Epoch  66 Batch    4/6   train_loss = 5.355
Epoch  75 Batch    0/6   train_loss = 5.287
Epoch  83 Batch    2/6   train_loss = 5.224
Epoch  91 Batch    4/6   train_loss = 5.218
Epoch 100 Batch    0/6   train_loss = 5.139
Epoch 108 Batch    2/6   train_loss = 5.039
Epoch 116 Batch    4/6   train_loss = 5.055
Epoch 125 Batch    0/6   train_loss = 4.960
Epoch 133 Batch    2/6   train_loss = 4.873
Epoch 141 Batch    4/6   train_loss = 4.907
Epoch 150 Batch    0/6   train_loss = 4.797
Epoch 158 Batch    2/6   train_loss = 4.704
Epoch 166 Batch    4/6   train_loss = 4.768
Epoch 175 Batch    0/6   train_loss = 4.617
Epoch 183 Batch    2/6   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [33]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [32]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)

tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [35]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    
    pr = np.argmax(probabilities)
    nextWord = int_to_vocab[pr]    
    return nextWord


tests.test_pick_word(pick_word)

Tests Passed


In [36]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(gets idea) isn't my car more!
moe_szyslak: okay.
kent_brockman: yeah!" i never see....
moe_szyslak: world, uh, would i ready to real life.
homer_simpson: i guess i got a name one of car better.
homer_simpson: well. you can just doin' right, things, kid like the money.
moe_szyslak: okay?
homer_simpson:(sobs) come on buy, but i was play home every day you someone a world!
homer_simpson:(ready) he's old!
moe_szyslak: aw, come on drink! can not doin' you guys-- duffman is a old old years it's back.
barney_gumble: uh. it's a next idea been here. just looks been this kid at play.
lenny_leonard: actually i play a little money?
chief_wiggum: well, if yeah god! he's just down your old japanese money, i'm looking about something clean don't tell ya little mr. smithers, play..... if you're happy to get it more bucks... and on
